In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
from datasets import load_dataset

dataset = load_dataset('/content/gdrive/MyDrive/Sentiment Analysis/vietnamese_students_feedback.py')

/usr/local/lib/python3.10/dist-packages/datasets/load.py:926: FutureWarning: The repository for vietnamese_students_feedback contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at /content/gdrive/MyDrive/Sentiment Analysis/vietnamese_students_feedback.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [7]:
import pandas as pd
df_test = pd.DataFrame(dataset['test'])
#df_test.loc[ : , 'sentence':'topic'] = df_test.loc[: , 'sentence':'topic'].str.lower()
df_train = pd.DataFrame(dataset['train'])
#df_train[['sentence', 'sentiment', 'topic']] = df_train[['sentence', 'sentiment', 'topic']].str.lower()
df_valid = pd.DataFrame(dataset['validation'])
# df_valid[['sentence', 'sentiment', 'topic']] = df_valid[['sentence', 'sentiment', 'topic']].str.lower()

In [8]:
!pip install vncorenlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645932 sha256=a9080367c42015fe38e895d68130fb56f38c29b6a8a839a3ff318a2853d216cc
  Stored in directory: /root/.cache/pip/wheels/5d/d9/b3/41f6c6b1ab758561fd4aab55dc0480b9d7a131c6aaa573a3fa
Successfully built vncorenlp


In [9]:
!mkdir -p vncorenlp/models/wordsegmenter
!wget https://github.com/vncorenlp/VnCoreNLP/archive/refs/heads/master.zip
!unzip master.zip
!mv VnCoreNLP-master/VnCoreNLP-1.1.1.jar vncorenlp/
!mv VnCoreNLP-master/models/wordsegmenter/* vncorenlp/models/wordsegmenter/

--2024-03-25 17:08:27--  https://github.com/vncorenlp/VnCoreNLP/archive/refs/heads/master.zip
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/vncorenlp/VnCoreNLP/zip/refs/heads/master [following]
--2024-03-25 17:08:27--  https://codeload.github.com/vncorenlp/VnCoreNLP/zip/refs/heads/master
Resolving codeload.github.com (codeload.github.com)... 20.27.177.114
Connecting to codeload.github.com (codeload.github.com)|20.27.177.114|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [   <=>              ] 160.84M  25.2MB/s    in 8.7s    

2024-03-25 17:08:36 (18.6 MB/s) - ‘master.zip’ saved [168648997]

Archive:  master.zip
62bbc58fe5d113c898eae112656be97dcf50b3a0
   creating: VnCoreNLP-master/
  inflating: VnCoreNLP-master/LICENSE.m

In [11]:
from vncorenlp import VnCoreNLP
vncorenlp_file = 'vncorenlp/VnCoreNLP-1.1.1.jar'

In [12]:
def word_segment(text, vncorenlp):
    segmented_text = vncorenlp.tokenize(text)
    flattened = [item for sublist in segmented_text for item in sublist]
    return ' '.join(flattened)

In [13]:
# test function word_segment
text1 = 'Chủ tịch nước Võ Văn Thưởng từ chức'
with VnCoreNLP(vncorenlp_file, annotators="wseg", port=9000) as vncorenlp:
  text2=word_segment(text1, vncorenlp)
  print(text2)

Chủ_tịch nước Võ_Văn_Thưởng từ_chức


In [14]:
with VnCoreNLP(vncorenlp_file, annotators="wseg", port=9000) as vncorenlp:
  df_train['sentence'] = df_train['sentence'].apply(lambda x: word_segment(x, vncorenlp))
  df_test['sentence'] = df_test['sentence'].apply(lambda x: word_segment(x, vncorenlp))
  df_valid['sentence'] = df_valid['sentence'].apply(lambda x: word_segment(x, vncorenlp))

In [15]:
print (df_train, df_test, df_valid)

                                                sentence  sentiment  topic
0                              slide giáo_trình đầy_đủ .          2      1
1         nhiệt_tình giảng_dạy , gần_gũi với sinh_viên .          2      0
2                   đi học đầy_đủ full điểm chuyên_cần .          0      1
3      chưa áp_dụng công_nghệ_thông_tin và các thiết_...          0      0
4      thầy giảng bài hay , có nhiều bài_tập ví_dụ ng...          2      0
...                                                  ...        ...    ...
11421  chỉ vì môn game mà em học hai lần mà không qua...          0      1
11422                               em cảm_ơn cô nhiều .          2      0
11423                           giao bài_tập quá nhiều .          0      0
11424               giáo_viên dạy dễ hiểu , nhiệt_tình .          2      0
11425  gói gọn doubledot hay , tận_tình , phù_hợp với...          2      0

[11426 rows x 3 columns]                                                sentence  sentiment  topic


In [16]:
# stripping
train_dataset = df_train.loc[: , ['sentence', 'sentiment']]
test_dataset = df_test.loc[: , ['sentence', 'sentiment']]
valid_dataset = df_valid.loc[: , ['sentence', 'sentiment']]

In [17]:
print(train_dataset, test_dataset, valid_dataset)

                                                sentence  sentiment
0                              slide giáo_trình đầy_đủ .          2
1         nhiệt_tình giảng_dạy , gần_gũi với sinh_viên .          2
2                   đi học đầy_đủ full điểm chuyên_cần .          0
3      chưa áp_dụng công_nghệ_thông_tin và các thiết_...          0
4      thầy giảng bài hay , có nhiều bài_tập ví_dụ ng...          2
...                                                  ...        ...
11421  chỉ vì môn game mà em học hai lần mà không qua...          0
11422                               em cảm_ơn cô nhiều .          2
11423                           giao bài_tập quá nhiều .          0
11424               giáo_viên dạy dễ hiểu , nhiệt_tình .          2
11425  gói gọn doubledot hay , tận_tình , phù_hợp với...          2

[11426 rows x 2 columns]                                                sentence  sentiment
0                              nói tiếng anh lưu_loát .          2
1                       

In [21]:
train_dataset.to_csv('/content/gdrive/MyDrive/Sentiment Analysis/data_train.csv', index=False)
test_dataset.to_csv('/content/gdrive/MyDrive/Sentiment Analysis/data_test.csv', index=False)
valid_dataset.to_csv('/content/gdrive/MyDrive/Sentiment Analysis/data_valid.csv', index=False)